In [23]:
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D,MaxPooling1D,  Dropout
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D,BatchNormalization
from keras.models import Model, Sequential
from keras.layers import Convolution1D,GlobalMaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
import numpy as np
import sys
import os

In [2]:
#df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_balanced.csv"
df = pd.read_csv("/Users/harvey/Desktop/Data/restuarant_review_balanced.csv")
#df = pd.read_csv("/home/ec2-user/Data/restuarant_review_balanced.csv")

In [3]:
df.head()

,sentiment,stars,Processed_Reviews
0,0,1,place go hill clearly cut back staff food qual...
1,0,1,walk around friday afternoon sit table bar wal...
2,0,1,cannot believe things change years pick duck c...
3,1,5,cant really find anything wrong place pastas p...
4,1,4,great lunch today staff helpful assist selecti...


In [4]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

58.18184846606756

In [5]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

In [6]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train['sentiment']
word_index = tokenizer.word_index

In [18]:
embed_size = 128
model = Sequential()
model.add(Embedding(len(word_index)+1, embed_size,input_length = maxlen,trainable = False))
model.add(LSTM(32, return_sequences = True))
#model.add(LSTM(32, return_sequences = True))
#model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 130, 128)          18109312  
_________________________________________________________________
lstm_9 (LSTM)                (None, 130, 32)           20608     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 20)                660       
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 21        
Total params: 18,130,601
Trainable params: 21,289
Non-trainable params: 18,109,312
___________________________________________________________

In [20]:
batch_size = 512
epochs = 2
history = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 165222 samples, validate on 41306 samples
Epoch 1/2
165222/165222 [==============================] - 94s 571us/step - loss: 0.5552 - acc: 0.7132 - val_loss: 0.4769 - val_acc: 0.7735
Epoch 2/2
165222/165222 [==============================] - 92s 557us/step - loss: 0.4473 - acc: 0.7904 - val_loss: 0.4029 - val_acc: 0.8163


In [25]:
#################################################################
#Save train history as dict 
#################################################################

with open(r"/Users/harvey/Desktop/Data/rnn_vanilla_2_labels", "wb") as output_file:
    pickle.dump(history.history, output_file)



In [27]:
with open(r"/Users/harvey/Desktop/Data/rnn_vanilla_2_labels", "rb") as input_file:
    e = pickle.load(input_file)

In [28]:
e

{'acc': [0.7132161576401195, 0.7904092675857931],
 'loss': [0.5552156828529206, 0.4472561666203743],
 'val_acc': [0.7735437949113694, 0.8163220838663948],
 'val_loss': [0.47687648440142705, 0.40294187479069266]}

In [26]:
y_test = test["sentiment"]
list_sentences_test = test['Processed_Reviews']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
#confusion_matrix(y_pred, y_test)

accuracy :0.8108537341183762
F1-score: 0.8188596653930333
Confusion matrix:


array([[ True],
       [False]])